In [15]:
import pandas as pd 
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy.spatial.distance import cdist
import sklearn 
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import mutual_info_classif, SelectKBest
import xgboost as xgb
from xgboost import XGBClassifier, plot_importance

#data = pd.read_csv('train_data.csv') # load original dataset 

# split dataset into 75/25 train test split 
#train, test = sklearn.model_selection.train_test_split(data, test_size=.25, random_state=42, shuffle=True, stratify=data['Label'])
#train.to_csv('split_train_data.csv', index = False) # save train data for future use 
#test.to_csv('split_test_data.csv', index = False) # save test data for future use 

train_split = pd.read_csv('split_train_data.csv')




In [ ]:
corr = train_split.corr(method='pearson')

In [ ]:
print(corr.iloc[:,0])

G26                                   1.000000
G26_Target Sequence_in_SMILES_perc   -0.658644
G26_SMILES_in_Target Sequence_perc   -0.633055
G26_ARRO                              0.169460
G26_SMILES_base                       0.317016
                                        ...   
G10_fd_Target Sequence_base           0.159265
G10_std_SMILES_dist                   0.430174
G10_std_Target Sequence_dist          0.111489
KIBA                                  0.201958
Label                                 0.152989
Name: G26, Length: 338, dtype: float64


In [ ]:
print(data.shape)

(109479, 338)


In [16]:
train_split = train_split.drop('KIBA', axis = 1)
train, test = sklearn.model_selection.train_test_split(train_split, test_size=.25, random_state=42, shuffle=True, stratify=train_split['Label'])

In [19]:
from sklearn.feature_selection import f_classif

In [ ]:
from sklearn.feature_selection import f_regression

In [ ]:
from sklearn.feature_selection import mutual_info_regression

In [17]:
print(train_split.shape)

(82109, 337)


In [27]:
print(train)

            G26  ...  Label
62386  5.771787  ...  False
63350  5.051778  ...  False
8881   5.648391  ...  False
15956  5.139381  ...  False
56441  5.467752  ...  False
...         ...  ...    ...
17916  5.220579  ...  False
8252   5.163831  ...   True
59884  5.036404  ...  False
39791  6.872042  ...  False
6867   5.609244  ...   True

[61581 rows x 337 columns]


In [20]:
# Feature Selection 
X = train.drop('Label', axis = 1)
y = train['Label']

k_best = SelectKBest(f_classif, k=100) # might want to update feature selection method / change k val
train_fs = k_best.fit(X, y)
cols = k_best.get_support(indices=True)
train_fs = train.iloc[:,cols]

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
%cd ../

/content/drive/MyDrive/SYSC5405/Project


In [21]:
print(cols)

[  0  10  13  14  15  16  18  20  22  23  24  25  26  27  42  43  48  49
  50  52  54  56  57  58  60  62  64  65  66  67  68  69  70  71  76  78
 140 141 142 146 148 149 150 151 152 153 196 203 210 211 218 220 222 224
 225 232 234 238 239 244 246 248 250 252 253 258 259 260 262 264 266 267
 268 274 275 276 278 279 280 281 286 287 288 292 295 300 308 309 314 315
 316 318 320 321 322 323 328 330 332 334]


In [29]:
print(train_fs)

            G26  G26_fd_SMILES_base  ...  G10_fd_SMILES_base  G10_std_SMILES_dist
62386  5.771787            0.076806  ...           -0.039719            -0.656573
63350  5.051778           -0.079144  ...           -0.036837            -0.573908
8881   5.648391            0.050248  ...           -0.007045             0.467020
15956  5.139381           -0.028619  ...            0.067213             2.927458
56441  5.467752           -0.015724  ...            0.004252             0.826105
...         ...                 ...  ...                 ...                  ...
17916  5.220579            0.020019  ...            0.084854             3.423547
8252   5.163831           -0.124667  ...            0.000754             0.752722
59884  5.036404           -0.023823  ...           -0.043870            -0.802460
39791  6.872042            0.221234  ...            0.075078             3.385024
6867   5.609244            0.045507  ...           -0.051585            -0.988755

[61581 rows x 1

In [30]:
# Outlier Removal 

# box plot pre-OR ?
preOL_mean = train_fs.mean()
preOL_med = train_fs.median()

Q1 = train_fs.quantile(0.25)
Q3 = train_fs.quantile(0.75)

IQR = Q3-Q1

# this sets outliers to Nan
# train_or=train_fs[~((train_fs<(Q1-1.5*IQR)) | (train_fs>(Q3+1.5*IQR)))]

# this relaces outliers with the corresponding bound 
train_or = train_fs[(train_fs>(Q1-1.5*IQR))]
train_or = train_or.fillna(Q1-1.5*IQR)
train_or = train_or[(train_fs<(Q3+1.5*IQR))]
train_or = train_or.fillna(Q3+1.5*IQR)

postOL_mean = train_or.mean()
postOL_med = train_or.median()
# box plot post OR ?

OLdescriptor_frames = [preOL_mean, preOL_med, postOL_mean, postOL_med]
OLdescriptor_names = ['preOL_mean', 'preOL_med', 'postOL_mean', 'postOL_med']

OLdescriptor = pd.concat(OLdescriptor_frames, axis = 1)
OLdescriptor.columns = OLdescriptor_names
#print(OLdescriptor)

In [49]:
print(train_or)

            G26  G26_fd_SMILES_base  ...  G10_fd_SMILES_base  G10_std_SMILES_dist
62386  5.771787            0.076806  ...           -0.039719            -0.656573
63350  5.051778           -0.079144  ...           -0.036837            -0.573908
8881   5.648391            0.050248  ...           -0.007045             0.467020
15956  5.139381           -0.028619  ...            0.067213             2.927458
56441  5.467752           -0.015724  ...            0.004252             0.826105
...         ...                 ...  ...                 ...                  ...
17916  5.220579            0.020019  ...            0.084854             3.423547
8252   5.163831           -0.124667  ...            0.000754             0.752722
59884  5.036404           -0.023823  ...           -0.043870            -0.802460
39791  6.175706            0.104837  ...            0.075078             3.385024
6867   5.609244            0.045507  ...           -0.051585            -0.988755

[61581 rows x 1

In [32]:
from sklearn import preprocessing

In [65]:
train_norm = train_or.values

In [66]:
print(train_norm)

[[ 5.77178669  0.0768065   2.17615677 ... -0.69013272 -0.03971912
  -0.65657302]
 [ 5.05177784 -0.07914385 -1.10831977 ... -0.6508944  -0.0368367
  -0.57390836]
 [ 5.64839125  0.05024753  0.67865392 ... -0.03981535 -0.00704541
   0.46701991]
 ...
 [ 5.03640413 -0.02382321 -1.74964522 ... -0.76514714 -0.04386997
  -0.80245971]
 [ 6.17570639  0.10483736  0.60116087 ...  0.1084824   0.07507801
   3.3850238 ]
 [ 5.60924435  0.04550669  0.69142603 ... -0.80957877 -0.05158534
  -0.98875548]]


In [67]:
min_max_scaler = preprocessing.MinMaxScaler()

In [68]:
x_scaled = min_max_scaler.fit_transform(train_norm)

In [72]:
df = pd.DataFrame(x_scaled, columns=train_or.columns)

In [73]:
print(df)

            G26  G26_fd_SMILES_base  ...  G10_fd_SMILES_base  G10_std_SMILES_dist
0      0.697589            0.889907  ...            0.348102             0.323693
1      0.158524            0.277403  ...            0.361077             0.335169
2      0.605204            0.785595  ...            0.495174             0.479687
3      0.224112            0.475842  ...            0.829430             0.821281
4      0.469960            0.526489  ...            0.546028             0.529540
...         ...                 ...  ...                 ...                  ...
61576  0.284904            0.666870  ...            0.908838             0.890156
61577  0.242418            0.098607  ...            0.530282             0.519352
61578  0.147014            0.494678  ...            0.329418             0.303438
61579  1.000000            1.000000  ...            0.864832             0.884808
61580  0.575895            0.766975  ...            0.294689             0.277574

[61581 rows x 1

In [61]:
train_norm = preprocessing.Normalizer().fit(train_or)

In [63]:
print(train_norm.transform(train_or))

[[ 0.19753808  0.00262868  0.07447847 ... -0.02361963 -0.00135938
  -0.02247106]
 [ 0.16845295 -0.00263907 -0.03695723 ... -0.02170426 -0.00122833
  -0.01913714]
 [ 0.18715873  0.00166495  0.02248711 ... -0.00131928 -0.00023345
   0.01547465]
 ...
 [ 0.16881012 -0.00079851 -0.05864458 ... -0.02564619 -0.00147043
  -0.02689683]
 [ 0.18062656  0.00306627  0.0175827  ...  0.00317288  0.00219588
   0.0990049 ]
 [ 0.18628563  0.0015113   0.02296258 ... -0.02688649 -0.00171317
  -0.03283703]]


In [74]:
scaled_df = pd.DataFrame(train_norm, columns=train_or.columns)

In [75]:
print(scaled_df)

            G26  G26_fd_SMILES_base  ...  G10_fd_SMILES_base  G10_std_SMILES_dist
0      5.771787            0.076806  ...           -0.039719            -0.656573
1      5.051778           -0.079144  ...           -0.036837            -0.573908
2      5.648391            0.050248  ...           -0.007045             0.467020
3      5.139381           -0.028619  ...            0.067213             2.927458
4      5.467752           -0.015724  ...            0.004252             0.826105
...         ...                 ...  ...                 ...                  ...
61576  5.220579            0.020019  ...            0.084854             3.423547
61577  5.163831           -0.124667  ...            0.000754             0.752722
61578  5.036404           -0.023823  ...           -0.043870            -0.802460
61579  6.175706            0.104837  ...            0.075078             3.385024
61580  5.609244            0.045507  ...           -0.051585            -0.988755

[61581 rows x 1

In [23]:
# Normalization 

# min max normalization 
train_mmnorm = train_or
for column in train_mmnorm.columns:
    col = train_mmnorm[column]
    num = col - col.min()
    denom = col.max()-col.min()
    train_mmnorm[column] = num/denom
print(train_mmnorm)

# z-score normalization 
#train_znorm = train_or
#for column in train_znorm.columns:
#    col = train_znorm[column]
#    num = col - col.mean()
#    denom = col.std()
#    train_znorm[column] = num/denom

#print(train_znorm)

            G26  G26_fd_SMILES_base  ...  G10_fd_SMILES_base  G10_std_SMILES_dist
62386  0.697589            0.889907  ...            0.348102             0.323693
63350  0.158524            0.277403  ...            0.361077             0.335169
8881   0.605204            0.785595  ...            0.495174             0.479687
15956  0.224112            0.475842  ...            0.829430             0.821281
56441  0.469960            0.526489  ...            0.546028             0.529540
...         ...                 ...  ...                 ...                  ...
17916  0.284904            0.666870  ...            0.908838             0.890156
8252   0.242418            0.098607  ...            0.530282             0.519352
59884  0.147014            0.494678  ...            0.329418             0.303438
39791  1.000000            1.000000  ...            0.864832             0.884808
6867   0.575895            0.766975  ...            0.294689             0.277574

[61581 rows x 1

In [ ]:
# k means classifier 
k_means = KMeans(init='k-means++', n_clusters=3, n_init=10) # n_clusters based on elbow plots! 
km_classiier = k_means.fit(train_mmnorm)

km_train = k_means.predict(train_mmnorm)

train_cluster = train_mmnorm
train_cluster['Cluster'] = list(km_train)
train_cluster['Label'] = y # if outliers are handeled differently (dropped) this will break 

In [ ]:
# for test set 
X = test.drop('Label', axis = 1)
y = test['Label']

# feature selection fron train
test_fs = k_best.fit(X, y)
test_fs = train.iloc[:,cols]

# outlier replacement using train set 
#test_or = test_fs[(test_fs>(Q1-1.5*IQR))]
#test_or = test_or.fillna(Q1-1.5*IQR)
#test_or = test_or[(test_fs<(Q3+1.5*IQR))]
#test_or = test_or.fillna(Q3+1.5*IQR)

# normalization?
# create test_mmnorm

km_test = k_means.predict(test_mmnorm)

test_cluster = test_mmnorm
test_cluster['Cluster'] = list(km_test)
test_cluster['Label'] = y

cluster1 = test_cluster[test_cluster['Cluster']==0]
yc1 = cluster1['Label']
Xc1 = cluster1.drop(['Label', 'Cluster'], axis = 1)
xgb_c1test = xgb.DMatrix(Xc1, label=yc1)

xgb_c1model.predict() 
#etccccc